### Lecture 4

- Exercise 1. Geometry construction
- Exercise 2. Coordinate transformation
- Exercise 3. Spatial index

In [ ]:
# download the data needed for this notebook
!rm -rf map_data && mkdir map_data

# road network
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/berkeley_nodes.csv" -O map_data/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/berkeley_links.csv" -O map_data/berkeley_links.csv

# evacuation zone shapefile
!rm -rf map_data/evacuation_zone && mkdir map_data/evacuation_zone
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/evacuation_zone/evacuation_zone.shp" -O map_data/evacuation_zone/evacuation_zone.shp
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/evacuation_zone/evacuation_zone.shx" -O map_data/evacuation_zone/evacuation_zone.shx
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/evacuation_zone/evacuation_zone.dbf" -O map_data/evacuation_zone/evacuation_zone.dfb
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/evacuation_zone/evacuation_zone.prj" -O map_data/evacuation_zone/evacuation_zone.prj

# parcel map shapefile
!rm -rf map_data/berkeley_parcels && mkdir map_data/berkeley_parcels
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/berkeley_parcels/parcels.shp" -O map_data/berkeley_parcels/parcels.shp
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/berkeley_parcels/parcels.shx" -O map_data/berkeley_parcels/parcels.shx
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/berkeley_parcels/parcels.dbf" -O map_data/berkeley_parcels/parcels.dfb
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/lecture_4/map_data/berkeley_parcels/parcels.prj" -O map_data/berkeley_parcels/parcels.prj

In [ ]:
# install library: geopandas for spatial data analysis. Shapely will be installed during the installation of geopandas.
!sudo apt install libspatialindex-dev
!pip install rtree pygeos geopandas

### Here is the beginning of the Python exercise.

In [ ]:
# import some general libraries
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

### Exercise 1. Geometry

In [ ]:
# import shapely for handling geometry data
from shapely.wkt import loads
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# load and display a point
point1 = loads('Point (30 20)')
point1

In [ ]:
# load and display a line string
line1 = loads("LINESTRING (10 10, 30 20, 40 40)")
line1

In [ ]:
# load and display a polygon
polygon1 = loads("POLYGON ((40 40, 10 30, 10 10, 30 20, 40 40))")
polygon1

In [ ]:
# exercise 1: what is the wkt of a circle?
# create a circle by drawing a buffer around a point
buffer1 = point1.buffer(10)
display(buffer1)
print(type(buffer1))
print(buffer1.wkt)

### Exercise 2. Coordinate Reference System (CRS)

In [ ]:
# import geopandas for pandas-like spatial data handling
import pandas as pd
import geopandas as gpd

In [ ]:
# read links data
links_df = pd.read_csv('map_data/berkeley_links.csv')
links_df.head()

In [ ]:
# construct a geodataframe from the pandas dataframe, a coordinate reference system (crs) code and WKT text strings
links_gdf = gpd.GeoDataFrame(links_df, crs='epsg:4326', 
                             geometry=links_df['geometry'].map(loads))

In [ ]:
# extract the geometry of the link 161 in the Geometric Coordinate System (lat/long), name it "link2_geometric"
link2_geographic = links_gdf.loc[links_df['link_id']==161].iloc[0].geometry
print( list(link2_geographic.coords) )
print( link2_geographic.length ) # this is wrong to calculate length!!
display(links_gdf.loc[links_df['link_id']==161].iloc[0].geometry)

In [ ]:
# project link 161 into the pseudo-mercator projection (EPSG:3857) or UTM Zone 10N projection (EPSG:26910). The units of coordinates in both projected coordinate systems are meters.
link2_mercator = links_gdf.to_crs(3857).loc[links_df['link_id']==161].iloc[0].geometry
print('Length of link 161 in Mercator projection is {} m'.format(link2_mercator.length))
link2_utm = links_gdf.to_crs(26910).loc[links_df['link_id']==161].iloc[0].geometry
print('Length of link 161 in UTM Zone 10N projection is {} m'.format(link2_utm.length))

In [ ]:
# we can also calculate it by hand
[(x1, y1), (x2, y2), (x3, y3)] = list(link2_mercator.coords)
link2_mercator_length = np.sqrt((y2-y1)**2 + (x2-x1)**2) + np.sqrt((y3-y2)**2 + (x3-x2)**2)
print('Length of link 161 in Mercator projection is {} m'.format(link2_mercator_length))

[(x1, y1), (x2, y2), (x3, y3)] = list(link2_utm.coords)
link2_utm_length = np.sqrt((y2-y1)**2 + (x2-x1)**2) + np.sqrt((y3-y2)**2 + (x3-x2)**2)
print('Length of link 161 in UTM Zone 10N projection is {} m'.format(link2_utm_length))

### Exercise 3. Spatial index

In [ ]:
# import modules.
# `time`: time the execution of the code.
# `shapely.ops.polygonize`: construct polygons from lines.
# `scipy.spatial.Voronoi`: make the Voronoi diagram.
import time
import shapely.ops
from scipy.spatial import Voronoi, voronoi_plot_2d

In [ ]:
# read the parcel data
berkeley_parcels = gpd.read_file('map_data/berkeley_parcels/parcels.shp')

# read the node data and convert it into a geodataframe
berkeley_nodes = pd.read_csv('map_data/berkeley_nodes.csv')
berkeley_nodes = gpd.GeoDataFrame(berkeley_nodes, crs='epsg:4326', geometry=[Point(xy) for xy in zip(berkeley_nodes.lon, berkeley_nodes.lat)])

# read the evacuation zone
evacuation_zone = gpd.read_file('map_data/evacuation_zone/evacuation_zone.shp')

In [ ]:
# plot figure. This may take some time.
fig, ax = plt.subplots(1, 2, figsize=(15,15))
berkeley_parcels.to_crs(26910).plot(ax = ax[0])
berkeley_nodes.to_crs(26910).plot(ax = ax[0], color='red')
evacuation_zone.to_crs(26910).plot(ax = ax[0], facecolor='none', edgecolor='orange')
berkeley_parcels.to_crs(26910).plot(ax = ax[1])
berkeley_nodes.to_crs(26910).plot(ax = ax[1], color='red')
ax[1].set_xlim([564500, 565500])
ax[1].set_ylim([4194000, 4195000])

In [ ]:
# add Voronoi diagram
# based on: https://gis.stackexchange.com/questions/337561/making-polygon-for-every-point-in-set-using-voronois
x = berkeley_nodes.geometry.x.values
y = berkeley_nodes.geometry.y.values
coords = np.vstack((x, y)).T
vor = Voronoi(coords)

lines = [shapely.geometry.LineString(vor.vertices[line]) for line in 
    vor.ridge_vertices if -1 not in line]
polys = shapely.ops.polygonize(lines)
voronois = gpd.GeoDataFrame(geometry=gpd.GeoSeries(polys), crs='epsg:4326').to_crs(26910)

voronois = gpd.overlay(voronois, evacuation_zone.to_crs(26910))
voronois['id'] = np.arange(voronois.shape[0])
voronois.head()

In [ ]:
# plot voronois. This again may take some time.
fig, ax = plt.subplots(1, 2, figsize=(15,15))

# whole plot
berkeley_parcels.to_crs(26910).plot(ax = ax[0])
voronois.to_crs(26910).plot(ax = ax[0], facecolor='none', edgecolor='red')
evacuation_zone.to_crs(26910).plot(ax = ax[0], facecolor='none', edgecolor='orange')

# zoom in
berkeley_parcels.to_crs(26910).plot(ax = ax[1])
berkeley_nodes.to_crs(26910).plot(ax = ax[1], color='red')
voronois.to_crs(26910).plot(ax = ax[1], facecolor='none', edgecolor='red')
ax[1].set_xlim([564500, 565500])
ax[1].set_ylim([4194000, 4195000])

In [ ]:
# extract the parcel centroid of each parcel
berkeley_parcels['centroid'] = berkeley_parcels.apply(lambda x: x['geometry'].centroid, axis=1)
berkeley_parcels_centroid = berkeley_parcels.set_geometry('centroid').to_crs(26910)

In [ ]:
# method 1: without spatial indexing. Possibly Colab is better than the test computer. The time difference with and without spatial index is not different much.
t0 = time.time()
parcels_per_node_1 = dict()

for voronoi_zone in voronois.itertuples():
    voronoi_gemetry = getattr(voronoi_zone, 'geometry')
    parcels_evacuate = berkeley_parcels_centroid[
        berkeley_parcels_centroid.intersects(voronoi_gemetry)]
    parcels_per_node_1[getattr(voronoi_zone, 'id')] = parcels_evacuate.shape[0]

print('It takes {:.2f} seconds to compute without spatial index'.format(time.time()-t0))

In [ ]:
# method 2: with spatial indexing
# based on: https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
t1 = time.time()
parcels_sindex = berkeley_parcels_centroid.sindex
parcels_per_node_2 = dict()

for voronoi_zone in voronois.itertuples():
    voronoi_gemetry = getattr(voronoi_zone, 'geometry')
    coarse_parcel_ids = list(parcels_sindex.intersection(voronoi_gemetry.bounds))
    coarse_parcels = berkeley_parcels_centroid.iloc[coarse_parcel_ids]
    parcels_evacuate = coarse_parcels[coarse_parcels.intersects(voronoi_gemetry)]
    parcels_per_node_2[getattr(voronoi_zone, 'id')] = parcels_evacuate.shape[0]

print('It takes {:.2f} seconds to compute with spatial index'.format(time.time()-t1))

equal_items = {k: parcels_per_node_1[k] for k in parcels_per_node_1 if k in parcels_per_node_2 and parcels_per_node_1[k] == parcels_per_node_2[k]}
print('There are {} voronoi shapes in method 1, {} in method 2, {} elements are equal'.format(len(parcels_per_node_1), len(parcels_per_node_2), len(equal_items)))

### backup materials

In [ ]:
# a hypothesized fire location
fire = gpd.GeoDataFrame(pd.DataFrame({'id': [0], 'name': ['ignition']}), crs='epsg:4326', geometry=[Point((-122.2502, 37.9021))])

# distance band to the hypothesized fire location
fire_propagation = pd.DataFrame([[distance, fire.to_crs(26910).iloc[0].geometry.buffer(distance*1609)] for distance in np.arange(0.2, 5, 0.2)], columns=['distance', 'geometry'])
fire_propagation = gpd.GeoDataFrame(fire_propagation, crs='epsg:26910', geometry=fire_propagation['geometry'])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
berkeley_parcels.to_crs(26910).plot(ax = ax)
fire_propagation.plot(ax = ax, facecolor='none', edgecolor='orange')

In [ ]:
# extract the parcel centroid
berkeley_parcels['centroid'] = berkeley_parcels.apply(lambda x: x['geometry'].centroid, axis=1)
berkeley_parcels_centroid = berkeley_parcels.set_geometry('centroid').to_crs(26910)

In [ ]:
# count the numbers of parcels within each distance band to the fire
# method 1: without spatial indexing
t0 = time.time()

for buffer_zone in fire_propagation.itertuples():
    t_i = time.time()
    parcels_evacuate = berkeley_parcels_centroid[
        berkeley_parcels_centroid.intersects(
            getattr(buffer_zone, 'geometry'))]
    print('{} parcels within {:.2f} miles to the fire location, computed in {:.3f} seconds.'.format(parcels_evacuate.shape[0], getattr(buffer_zone, 'distance'), time.time()-t_i))

print('It takes {:.2f} seconds to compute'.format(time.time()-t0))

In [ ]:
# method 2: with spatial indexing
# based on: https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
t1 = time.time()
parcels_sindex = berkeley_parcels_centroid.sindex

for buffer_zone in fire_propagation.itertuples():
    t_i = time.time()
    buffer_gemetry = getattr(buffer_zone, 'geometry')
    coarse_parcel_ids = list(parcels_sindex.intersection(buffer_gemetry.bounds))
    coarse_parcels = berkeley_parcels_centroid.iloc[coarse_parcel_ids]
    parcels_evacuate = coarse_parcels[coarse_parcels.intersects(buffer_gemetry)]
    print('{} parcels within {:.2f} miles to the fire location, computed in {:.3f} seconds.'.format(parcels_evacuate.shape[0], getattr(buffer_zone, 'distance'), time.time()-t_i))

print('It takes {:.2f} seconds to compute'.format(time.time()-t1))